In [1]:
import tensorflow as tf
print(tf.test.gpu_device_name())

/device:GPU:0


In [2]:
!nvidia-smi

Tue May  5 09:51:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    32W / 250W |    353MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
import os

# choose a local (colab) directory to store the data.
local_root_path = os.path.expanduser("~/data")
try:
  os.makedirs(local_root_path)
except: pass

def ListFolder(google_drive_id, destination):
  file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % google_drive_id}).GetList()
  counter = 0
  for f in file_list:
    # If it is a directory then, create the dicrectory and upload the file inside it
    if f['mimeType']=='application/vnd.google-apps.folder': 
      folder_path = os.path.join(destination, f['title'])
      os.makedirs(folder_path)
      print('creating directory {}'.format(folder_path))
      ListFolder(f['id'], folder_path)
    else:
      fname = os.path.join(destination, f['title'])
      f_ = drive.CreateFile({'id': f['id']})
      f_.GetContentFile(fname)
      counter += 1
  print('{} files were uploaded in {}'.format(counter, destination))

ListFolder("1aLvYGyuY6xWLUssYRVyyk3EtPZodLUBR", local_root_path)

creating directory /root/data/mnist
4 files were uploaded in /root/data/mnist
creating directory /root/data/fashion
5 files were uploaded in /root/data/fashion
1 files were uploaded in /root/data


In [0]:
# Vorstellung: MNIST-Daten!
# http://yann.lecun.com/exdb/mnist/
# FashionMNIST: https://github.com/zalandoresearch/fashion-mnist

import gzip
import numpy as np
from keras.utils import to_categorical

def open_images(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=16)\
            .reshape(-1, 28, 28)\
            .astype(np.float32)


def open_labels(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=8)
    
X_train = open_images("/root/data/fashion/train-images-idx3-ubyte.gz")
y_train = open_labels("/root/data/fashion/train-labels-idx1-ubyte.gz")

X_test = open_images("/root/data/fashion/t10k-images-idx3-ubyte.gz")
y_test = open_labels("/root/data/fashion/t10k-labels-idx1-ubyte.gz")

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
# CNN!

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

model = Sequential()

model.add(Conv2D(30, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)))
model.add(Flatten())
model.add(Dense(10, activation="softmax"))

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(
    X_train.reshape(60000, 28, 28, 1),
    y_train,
    epochs=80,
    batch_size=1000)

Epoch 1/80
60000/60000 [==============================] - 1s 10us/step - loss: 44.8694 - accuracy: 0.6693
Epoch 2/80
60000/60000 [==============================] - 1s 9us/step - loss: 2.8741 - accuracy: 0.8208
Epoch 3/80
60000/60000 [==============================] - 1s 8us/step - loss: 0.5555 - accuracy: 0.8632
Epoch 4/80
60000/60000 [==============================] - 1s 9us/step - loss: 0.3067 - accuracy: 0.8925
Epoch 5/80
60000/60000 [==============================] - 1s 9us/step - loss: 0.2345 - accuracy: 0.9138
Epoch 6/80
60000/60000 [==============================] - 1s 9us/step - loss: 0.1983 - accuracy: 0.9264
Epoch 7/80
60000/60000 [==============================] - 1s 9us/step - loss: 0.1681 - accuracy: 0.9383
Epoch 8/80
60000/60000 [==============================] - 1s 9us/step - loss: 0.1508 - accuracy: 0.9443
Epoch 9/80
60000/60000 [==============================] - 1s 9us/step - loss: 0.1305 - accuracy: 0.9522
Epoch 10/80
60000/60000 [==============================] - 1s 